# math.SG : Récupération et formattage des données

_Remarque._ 
- Année 2019 : articles no 6402 à 6855 (454 articles) 
- 2009 : articles no 2451 à 2778 (327 articles) 
- total (jusqu'au 19 mai 2020) : 0 à 7045

In [1]:
import urllib.request as libreq
import xmltodict
import pandas as pd
from time import sleep

In [2]:
# pour les colonnes author/category
def nb(x):
    if type(x) == list: return len(x)
    else: return 1

def ODict2List(x,key):
    if type(x) == list: return ' | '.join([x[i][key] for i in range(len(x))])
    else: return x[key]
    
# transformer xml en dictionnaries    
def entry2dict(entry):
    return xmltodict.parse(entry, process_namespaces=True, namespaces=namespaces)['entry']

In [3]:
for start in [0,2000,4000,6000]:

    # Récupération sous la forme d'un fichier xml
    with libreq.urlopen(f'http://export.arxiv.org/api/query?search_query=cat:math.SG&start={start}&max_results=2000&sortBy=submittedDate&sortOrder=ascending') as url:
        r = url.read()
        
    # Enregistrer les fichiers xml (au cas où ... pas besoin de re-charger le serveur d'arXiv)
    file = open(f'tmp/arXiv_SG_{start}.xml', 'wb')
    file.write(r)
    file.close()
    
    # attendre 20 secondes entre chaque requête
    sleep(20)    

## Préparation des 4 fichiers

In [4]:
namespaces = {
    'http://arxiv.org/schemas/atom': None,
#    'http://a9.com/-/spec/opensearch/1.1/': None, # skip this namespace
#    'http://www.w3.org/2005/Atom' : None,
#    'http://a.com/': 'ns_a', # collapse "http://a.com/" -> "ns_a"
}

for start in [0,2000,4000,6000]:
    
    # sous la forme d'une unique chaîne de texte sans le "</feed>" de la fin
    with open(f'tmp/arXiv_SG_{str(start)}.xml') as fd:
        raw_string = fd.read()
    raw_string = raw_string[:-9]
    
    # transformation en listes d'entrées puis en liste de dictionnaires
    # penser à virer 1er élément de la liste qui n'est pas une entrée
    list_of_papers = raw_string.split('<entry>')
    list_of_papers = ["<entry>" + paper for paper in list_of_papers[1:]]
    list_of_dicts = [entry2dict(paper) for paper in list_of_papers]

    # transformation en dataframe
    arXiv_df = pd.DataFrame(list_of_dicts)
    arXiv_df = arXiv_df.drop(columns=['link','journal_ref', 'doi'])
    
    # remplace dictionnaire par unique entrée qui m'importe
    # garder uniquement id dans "id"
    arXiv_df['primary_category'] = arXiv_df['primary_category'].map(lambda dict: dict['@term'])
    arXiv_df['id'] = arXiv_df['id'].map(lambda id: id[21:])
    
    # Ajout cols : nb version finale, nb d'auteurs
    arXiv_df['final_version'] = arXiv_df['id'].map(lambda x: x[-1])
    arXiv_df['nb_authors'] = arXiv_df['author'].map(lambda x: nb(x))
    arXiv_df['nb_category'] = arXiv_df['category'].map(lambda x: nb(x))

    # transforme le ODict des auteurs en une string, separation "|"
    arXiv_df['list_authors'] = arXiv_df['author'].map(lambda x: ODict2List(x,'name'))
    arXiv_df = arXiv_df.drop(columns=['author'])
    arXiv_df['list_categories'] = arXiv_df['category'].map(lambda x: ODict2List(x,'@term'))
    arXiv_df = arXiv_df.drop(columns=['category'])
    
    # Enregistrer en csv
    arXiv_df.to_csv(f'tmp/arXiv_SG_{str(start)}_formatted.csv')

## On rassemble tout en un unique dataframe

In [5]:
df = []

for start in [0,2000,4000,6000]:
    df.append(pd.read_csv(f'tmp/arXiv_SG_{start}_formatted.csv'))
    
math_SG = pd.concat(df, ignore_index=True)

In [6]:
math_SG.tail(1)

,Unnamed: 0,id,updated,published,title,summary,primary_category,comment,final_version,nb_authors,nb_category,list_authors,list_categories
7066,1066,2006.03521v1,2020-06-05T15:56:26Z,2020-06-05T15:56:26Z,L-space knots have no essential Conway spheres,We prove that L-space knots do not have essent...,math.GT,"22 pages, 11 color figures created with PSTric...",1,3,3,Tye Lidman | Allison H. Moore | Claudius Zibro...,math.GT | math.QA | math.SG


In [7]:
math_SG.to_csv('data/arXiv_SG_total_formatted.csv')